Flow of the developed model:
1) Define the 16 independent variables in the positive electrode 

2) Define parameters

3) Defining intermediate variables and governing equations

4) Add boundary conditions and initial conditions

5) Enter a set of parameter values

6) Define geometry, mesh type and discretize the model

7) Solve the model

In [1]:
#%pip install pybamm -q #for installing pybamm if not installed
import pybamm
pybamm.set_logging_level("DEBUG")
import numpy as np
import matplotlib.pyplot as plt

model1=pybamm.BaseModel(name = "1D LiS (without separator)")

Separator = 1, Positive electrode = 2

i: 1=Li$^+$, 2=S$_{8(l)}$, 3=S$_8^{2-}$, 4=S$_6^{2-}$, 5=S$_4^{2-}$, 6=S$_2^{2-}$, 7=S$^{2-}$, 8=A$^-$

k: 1=S$_{8(s)}$, 2=Li$_2$S$_{8(s)}$, 3=Li$_2$S$_{4(s)}$, 4=Li$_2$S$_{2(s)}$, 5=Li$_2$S$_{(s)}$

j: 

1 = Li $\leftrightarrow$ Li$^+$ + e$^-$

2 = $\frac{1}{2}$S$_{8(l)}$ + e$^-$ $\leftrightarrow$ $\frac{1}{2}$S$_8^{2-}$

3 = $\frac{3}{2}$S$_8^{2-}$ + e$^-$ $\leftrightarrow$ 2S$_6^{2-}$

4 = S$_6^{2-}$ + e$^-$ $\leftrightarrow$ $\frac{3}{2}$S$_4^{2-}$

5 = $\frac{1}{2}$S$_4^{2-}$ + e$^-$ $\leftrightarrow$ S$_2^{2-}$

6 = $\frac{1}{2}$S$_2^{2-}$ + e$^-$ $\leftrightarrow$ S$^{2-}$

In [2]:
#Adding porosity and concentration variables (porosity*concentration taken as a variable with concentration defined separately)
#(C_i_2 == C_i in positive electrode, C_i_1 == C_i in separator)

C1=pybamm.Variable("Concentration of Li(+) in positive electrode [mol.m-3]",domain="positive electrode")
C2=pybamm.Variable("Concentration of S8(l) in positive electrode [mol.m-3]",domain="positive electrode")
C3=pybamm.Variable("Concentration of S8(2-) in positive electrode [mol.m-3]",domain="positive electrode")
C4=pybamm.Variable("Concentration of S6(2-) in positive electrode [mol.m-3]",domain="positive electrode")
C5=pybamm.Variable("Concentration of S4(2-) in positive electrode [mol.m-3]",domain="positive electrode")
C6=pybamm.Variable("Concentration of S2(2-) in positive electrode [mol.m-3]",domain="positive electrode")
C7=pybamm.Variable("Concentration of S(2-) in positive electrode [mol.m-3]",domain="positive electrode")
C8=pybamm.Variable("Concentration of A(-) in positive electrode [mol.m-3]",domain="positive electrode")

epsilon=pybamm.Variable("Porosity of positive electrode",domain="positive electrode")

In [3]:
#Adding the remaining variables

epsilon1=pybamm.Variable("Volume fraction of S8(s) precipitate in positive electrode",domain="positive electrode")
epsilon2=pybamm.Variable("Volume fraction of Li2S8(s) precipitate in positive electrode",domain="positive electrode")
epsilon3=pybamm.Variable("Volume fraction of Li2S4(s) precipitate in positive electrode",domain="positive electrode")
epsilon4=pybamm.Variable("Volume fraction of Li2S2(s) precipitate in positive electrode",domain="positive electrode")
epsilon5=pybamm.Variable("Volume fraction of Li2S(s) precipitate in positive electrode",domain="positive electrode")

phi1=pybamm.Variable("Solid phase potential in positive electrode [V]",domain="positive electrode")
phi2=pybamm.Variable("Liquid phase potential in positive electrode [V]",domain="positive electrode")

In [4]:
#Defining parameters

F=pybamm.Parameter("Faraday's constant [C.mol-1]")
R=pybamm.Parameter("Gas constant [J.K-1.mol-1]")
T=pybamm.Parameter("Temperature [K]")
b=pybamm.Parameter("Bruggeman coefficient")
a0=pybamm.Parameter("Initial value of specific area of positive electrode [m2.m-3]")
zeta=pybamm.Parameter("Morphology parameter")
sigma=pybamm.Parameter("Effective conductivity of solid phase of the positive electrode [S.m-1]")

Iapp=pybamm.Parameter("Externally applied current [A.m-2]") #External current (design parameter)

#z_i
z1=pybamm.Parameter("Charge number of Li(+)")
z2=pybamm.Parameter("Charge number of S8(l)")
z3=pybamm.Parameter("Charge number of S8(2-)")
z4=pybamm.Parameter("Charge number of S6(2-)")
z5=pybamm.Parameter("Charge number of S4(2-)")
z6=pybamm.Parameter("Charge number of S2(2-)")
z7=pybamm.Parameter("Charge number of S(2-)")
z8=pybamm.Parameter("Charge number of A(-)")

#D0_i
D01=pybamm.Parameter("Diffusion coefficient of Li(+) in bulk medium [m2.s-1]")
D02=pybamm.Parameter("Diffusion coefficient of S8(l) in bulk medium [m2.s-1]")
D03=pybamm.Parameter("Diffusion coefficient of S8(2-) in bulk medium [m2.s-1]")
D04=pybamm.Parameter("Diffusion coefficient of S6(2-) in bulk medium [m2.s-1]")
D05=pybamm.Parameter("Diffusion coefficient of S4(2-) in bulk medium [m2.s-1]")
D06=pybamm.Parameter("Diffusion coefficient of S2(2-) in bulk medium [m2.s-1]")
D07=pybamm.Parameter("Diffusion coefficient of S(2-) in bulk medium [m2.s-1]")
D08=pybamm.Parameter("Diffusion coefficient of A(-) in bulk medium [m2.s-1]")

#s_i_j
s11=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 1")
s12=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 2")
s13=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 3")
s14=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 4")
s15=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 5")
s16=pybamm.Parameter("Stoichiometric coefficient of Li(+) in reaction 6")
s21=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 1")
s22=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 2")
s23=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 3")
s24=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 4")
s25=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 5")
s26=pybamm.Parameter("Stoichiometric coefficient of S8(l) in reaction 6")
s31=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 1")
s32=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 2")
s33=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 3")
s34=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 4")
s35=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 5")
s36=pybamm.Parameter("Stoichiometric coefficient of S8(2-) in reaction 6")
s41=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 1")
s42=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 2")
s43=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 3")
s44=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 4")
s45=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 5")
s46=pybamm.Parameter("Stoichiometric coefficient of S6(2-) in reaction 6")
s51=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 1")
s52=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 2")
s53=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 3")
s54=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 4")
s55=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 5")
s56=pybamm.Parameter("Stoichiometric coefficient of S4(2-) in reaction 6")
s61=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 1")
s62=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 2")
s63=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 3")
s64=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 4")
s65=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 5")
s66=pybamm.Parameter("Stoichiometric coefficient of S2(2-) in reaction 6")
s71=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 1")
s72=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 2")
s73=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 3")
s74=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 4")
s75=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 5")
s76=pybamm.Parameter("Stoichiometric coefficient of S(2-) in reaction 6")
s81=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 1")
s82=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 2")
s83=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 3")
s84=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 4")
s85=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 5")
s86=pybamm.Parameter("Stoichiometric coefficient of A(-) in reaction 6")

#n_j
n1=pybamm.Parameter("Number of electrons transferred in reaction 1")
n2=pybamm.Parameter("Number of electrons transferred in reaction 2")
n3=pybamm.Parameter("Number of electrons transferred in reaction 3")
n4=pybamm.Parameter("Number of electrons transferred in reaction 4")
n5=pybamm.Parameter("Number of electrons transferred in reaction 5")
n6=pybamm.Parameter("Number of electrons transferred in reaction 6")

epsilon0=pybamm.Parameter("Initial value of porosity of positive electrode")

#i0_jref
i01ref=pybamm.Parameter("Exchange current density of reaction 1 at reference concentrations [A.m-2]")
i02ref=pybamm.Parameter("Exchange current density of reaction 2 at reference concentrations [A.m-2]")
i03ref=pybamm.Parameter("Exchange current density of reaction 3 at reference concentrations [A.m-2]")
i04ref=pybamm.Parameter("Exchange current density of reaction 4 at reference concentrations [A.m-2]")
i05ref=pybamm.Parameter("Exchange current density of reaction 5 at reference concentrations [A.m-2]")
i06ref=pybamm.Parameter("Exchange current density of reaction 6 at reference concentrations [A.m-2]")

#C_iref
C1ref=pybamm.Parameter("Reference concentration of Li(+) [mol.m-3]")
C2ref=pybamm.Parameter("Reference concentration of S8(l) [mol.m-3]")
C3ref=pybamm.Parameter("Reference concentration of S8(2-) [mol.m-3]")
C4ref=pybamm.Parameter("Reference concentration of S6(2-) [mol.m-3]")
C5ref=pybamm.Parameter("Reference concentration of S4(2-) [mol.m-3]")
C6ref=pybamm.Parameter("Reference concentration of S2(2-) [mol.m-3]")
C7ref=pybamm.Parameter("Reference concentration of S(2-) [mol.m-3]")
C8ref=pybamm.Parameter("Reference concentration of A(-) [mol.m-3]")

#alpha_aj
alpha_a1=pybamm.Parameter("Anodic transfer coefficient of reaction 1")
alpha_a2=pybamm.Parameter("Anodic transfer coefficient of reaction 2")
alpha_a3=pybamm.Parameter("Anodic transfer coefficient of reaction 3")
alpha_a4=pybamm.Parameter("Anodic transfer coefficient of reaction 4")
alpha_a5=pybamm.Parameter("Anodic transfer coefficient of reaction 5")
alpha_a6=pybamm.Parameter("Anodic transfer coefficient of reaction 6")

#alpha_cj
alpha_c1=pybamm.Parameter("Cathodic transfer coefficient of reaction 1")
alpha_c2=pybamm.Parameter("Cathodic transfer coefficient of reaction 2")
alpha_c3=pybamm.Parameter("Cathodic transfer coefficient of reaction 3")
alpha_c4=pybamm.Parameter("Cathodic transfer coefficient of reaction 4")
alpha_c5=pybamm.Parameter("Cathodic transfer coefficient of reaction 5")
alpha_c6=pybamm.Parameter("Cathodic transfer coefficient of reaction 6")

#Utheta_j
Utheta1=pybamm.Parameter("Standard OCP of reaction 1 [V]")
Utheta2=pybamm.Parameter("Standard OCP of reaction 2 [V]")
Utheta3=pybamm.Parameter("Standard OCP of reaction 3 [V]")
Utheta4=pybamm.Parameter("Standard OCP of reaction 4 [V]")
Utheta5=pybamm.Parameter("Standard OCP of reaction 5 [V]")
Utheta6=pybamm.Parameter("Standard OCP of reaction 6 [V]")

#gamma_i_k
gamma11=pybamm.Parameter("Number of Li(+) produced by dissociation of S8(s)")
gamma12=pybamm.Parameter("Number of Li(+) produced by dissociation of Li2S8(s)")
gamma13=pybamm.Parameter("Number of Li(+) produced by dissociation of Li2S4(s)")
gamma14=pybamm.Parameter("Number of Li(+) produced by dissociation of Li2S2(s)")
gamma15=pybamm.Parameter("Number of Li(+) produced by dissociation of Li2S(s)")
gamma21=pybamm.Parameter("Number of S8(l) produced by dissociation of S8(s)")
gamma22=pybamm.Parameter("Number of S8(l) produced by dissociation of Li2S8(s)")
gamma23=pybamm.Parameter("Number of S8(l) produced by dissociation of Li2S4(s)")
gamma24=pybamm.Parameter("Number of S8(l) produced by dissociation of Li2S2(s)")
gamma25=pybamm.Parameter("Number of S8(l) produced by dissociation of Li2S(s)")
gamma31=pybamm.Parameter("Number of S8(2-) produced by dissociation of S8(s)")
gamma32=pybamm.Parameter("Number of S8(2-) produced by dissociation of Li2S8(s)")
gamma33=pybamm.Parameter("Number of S8(2-) produced by dissociation of Li2S4(s)")
gamma34=pybamm.Parameter("Number of S8(2-) produced by dissociation of Li2S2(s)")
gamma35=pybamm.Parameter("Number of S8(2-) produced by dissociation of Li2S(s)")
gamma41=pybamm.Parameter("Number of S6(2-) produced by dissociation of S8(s)")
gamma42=pybamm.Parameter("Number of S6(2-) produced by dissociation of Li2S8(s)")
gamma43=pybamm.Parameter("Number of S6(2-) produced by dissociation of Li2S4(s)")
gamma44=pybamm.Parameter("Number of S6(2-) produced by dissociation of Li2S2(s)")
gamma45=pybamm.Parameter("Number of S6(2-) produced by dissociation of Li2S(s)")
gamma51=pybamm.Parameter("Number of S4(2-) produced by dissociation of S8(s)")
gamma52=pybamm.Parameter("Number of S4(2-) produced by dissociation of Li2S8(s)")
gamma53=pybamm.Parameter("Number of S4(2-) produced by dissociation of Li2S4(s)")
gamma54=pybamm.Parameter("Number of S4(2-) produced by dissociation of Li2S2(s)")
gamma55=pybamm.Parameter("Number of S4(2-) produced by dissociation of Li2S(s)")
gamma61=pybamm.Parameter("Number of S2(2-) produced by dissociation of S8(s)")
gamma62=pybamm.Parameter("Number of S2(2-) produced by dissociation of Li2S8(s)")
gamma63=pybamm.Parameter("Number of S2(2-) produced by dissociation of Li2S4(s)")
gamma64=pybamm.Parameter("Number of S2(2-) produced by dissociation of Li2S2(s)")
gamma65=pybamm.Parameter("Number of S2(2-) produced by dissociation of Li2S(s)")
gamma71=pybamm.Parameter("Number of S(2-) produced by dissociation of S8(s)")
gamma72=pybamm.Parameter("Number of S(2-) produced by dissociation of Li2S8(s)")
gamma73=pybamm.Parameter("Number of S(2-) produced by dissociation of Li2S4(s)")
gamma74=pybamm.Parameter("Number of S(2-) produced by dissociation of Li2S2(s)")
gamma75=pybamm.Parameter("Number of S(2-) produced by dissociation of Li2S(s)")
gamma81=pybamm.Parameter("Number of A(-) produced by dissociation of S8(s)")
gamma82=pybamm.Parameter("Number of A(-) produced by dissociation of Li2S8(s)")
gamma83=pybamm.Parameter("Number of A(-) produced by dissociation of Li2S4(s)")
gamma84=pybamm.Parameter("Number of A(-) produced by dissociation of Li2S2(s)")
gamma85=pybamm.Parameter("Number of A(-) produced by dissociation of Li2S(s)")

#k_k
k1=pybamm.Parameter("Rate constant of S8(s) [s-1]")
k2=pybamm.Parameter("Rate constant of Li2S8(s) [m6.mol2.s-1]")
k3=pybamm.Parameter("Rate constant of Li2S4(s) [m6.mol2.s-1]")
k4=pybamm.Parameter("Rate constant of Li2S2(s) [m6.mol2.s-1]")
k5=pybamm.Parameter("Rate constant of Li2S(s) [m6.mol2.s-1]")

#ksp_k
ksp1=pybamm.Parameter("Solubility product of S8(s) [mol.m-3]")
ksp2=pybamm.Parameter("Solubility product of Li2S8(s) [mol3.m-9]")
ksp3=pybamm.Parameter("Solubility product of Li2S4(s) [mol3.m-9]")
ksp4=pybamm.Parameter("Solubility product of Li2S2(s) [mol3.m-9]")
ksp5=pybamm.Parameter("Solubility product of Li2S(s) [mol3.m-9]")

#V_k
V1=pybamm.Parameter("Molar volume of S8(s) [m3.mol-1]")
V2=pybamm.Parameter("Molar volume of Li2S8(s) [m3.mol-1]")
V3=pybamm.Parameter("Molar volume of Li2S4(s) [m3.mol-1]")
V4=pybamm.Parameter("Molar volume of Li2S2(s) [m3.mol-1]")
V5=pybamm.Parameter("Molar volume of Li2S(s) [m3.mol-1]")

#Thickness parameter
Lc=pybamm.Parameter("Thickness of positive electrode [m]")

#U_jref [V]
U1ref=pybamm.Parameter("Reference OCP of reaction 1 [V]")
U2ref=pybamm.Parameter("Reference OCP of reaction 2 [V]")
U3ref=pybamm.Parameter("Reference OCP of reaction 3 [V]")
U4ref=pybamm.Parameter("Reference OCP of reaction 4 [V]")
U5ref=pybamm.Parameter("Reference OCP of reaction 5 [V]")
U6ref=pybamm.Parameter("Reference OCP of reaction 6 [V]")

In [5]:
#Defining functional parameters

#D_i in positive electrode
inputsD={"Porosity of positive electrode":epsilon}
D1=pybamm.FunctionParameter("Diffusion coefficient of Li(+) in porous medium in positive electrode [m2.s-1]",inputsD)
D2=pybamm.FunctionParameter("Diffusion coefficient of S8(l) in porous medium in positive electrode [m2.s-1]",inputsD)
D3=pybamm.FunctionParameter("Diffusion coefficient of S8(2-) in porous medium in positive electrode [m2.s-1]",inputsD)
D4=pybamm.FunctionParameter("Diffusion coefficient of S6(2-) in porous medium in positive electrode [m2.s-1]",inputsD)
D5=pybamm.FunctionParameter("Diffusion coefficient of S4(2-) in porous medium in positive electrode [m2.s-1]",inputsD)
D6=pybamm.FunctionParameter("Diffusion coefficient of S2(2-) in porous medium in positive electrode [m2.s-1]",inputsD)
D7=pybamm.FunctionParameter("Diffusion coefficient of S(2-) in porous medium in positive electrode [m2.s-1]",inputsD)
D8=pybamm.FunctionParameter("Diffusion coefficient of A(-) in porous medium in positive electrode [m2.s-1]",inputsD)

#Specific area of positive electrode
inputsa={"Porosity of positive electrode":epsilon}
a=pybamm.FunctionParameter("Specific surface area of positive electrode [m2.m-3]",inputsa)

Now, with all parameters defined, we proceed towards writing down governing equations followed by boundary conditions and initial conditions. We start by defining useful intermediate variables.

In [6]:
##U_jref [V]
#U_1ref = Utheta_1 - R*T/(n_1*F)*((s_1_1*pybamm.log(C_1ref/1000, base='e')) + (s_2_1*pybamm.log(C_2ref/1000, base='e')) + (s_3_1*pybamm.log(C_3ref/1000, base='e')) + (s_4_1*pybamm.log(C_4ref/1000, base='e')) + (s_5_1*pybamm.log(C_5ref/1000, base='e')) + (s_6_1*pybamm.log(C_6ref/1000, base='e')) + (s_7_1*pybamm.log(C_7ref/1000, base='e')) + (s_8_1*pybamm.log(C_8ref/1000, base='e')))
#U_2ref = Utheta_2 - R*T/(n_2*F)*((s_1_2*pybamm.log(C_1ref/1000, base='e')) + (s_2_2*pybamm.log(C_2ref/1000, base='e')) + (s_3_2*pybamm.log(C_3ref/1000, base='e')) + (s_4_2*pybamm.log(C_4ref/1000, base='e')) + (s_5_2*pybamm.log(C_5ref/1000, base='e')) + (s_6_2*pybamm.log(C_6ref/1000, base='e')) + (s_7_2*pybamm.log(C_7ref/1000, base='e')) + (s_8_2*pybamm.log(C_8ref/1000, base='e')))
#U_3ref = Utheta_3 - R*T/(n_3*F)*((s_1_3*pybamm.log(C_1ref/1000, base='e')) + (s_2_3*pybamm.log(C_2ref/1000, base='e')) + (s_3_3*pybamm.log(C_3ref/1000, base='e')) + (s_4_3*pybamm.log(C_4ref/1000, base='e')) + (s_5_3*pybamm.log(C_5ref/1000, base='e')) + (s_6_3*pybamm.log(C_6ref/1000, base='e')) + (s_7_3*pybamm.log(C_7ref/1000, base='e')) + (s_8_3*pybamm.log(C_8ref/1000, base='e')))
#U_4ref = Utheta_4 - R*T/(n_4*F)*((s_1_4*pybamm.log(C_1ref/1000, base='e')) + (s_2_4*pybamm.log(C_2ref/1000, base='e')) + (s_3_4*pybamm.log(C_3ref/1000, base='e')) + (s_4_4*pybamm.log(C_4ref/1000, base='e')) + (s_5_4*pybamm.log(C_5ref/1000, base='e')) + (s_6_4*pybamm.log(C_6ref/1000, base='e')) + (s_7_4*pybamm.log(C_7ref/1000, base='e')) + (s_8_4*pybamm.log(C_8ref/1000, base='e')))
#U_5ref = Utheta_5 - R*T/(n_5*F)*((s_1_5*pybamm.log(C_1ref/1000, base='e')) + (s_2_5*pybamm.log(C_2ref/1000, base='e')) + (s_3_5*pybamm.log(C_3ref/1000, base='e')) + (s_4_5*pybamm.log(C_4ref/1000, base='e')) + (s_5_5*pybamm.log(C_5ref/1000, base='e')) + (s_6_5*pybamm.log(C_6ref/1000, base='e')) + (s_7_5*pybamm.log(C_7ref/1000, base='e')) + (s_8_5*pybamm.log(C_8ref/1000, base='e')))
#U_6ref = Utheta_6 - R*T/(n_6*F)*((s_1_6*pybamm.log(C_1ref/1000, base='e')) + (s_2_6*pybamm.log(C_2ref/1000, base='e')) + (s_3_6*pybamm.log(C_3ref/1000, base='e')) + (s_4_6*pybamm.log(C_4ref/1000, base='e')) + (s_5_6*pybamm.log(C_5ref/1000, base='e')) + (s_6_6*pybamm.log(C_6ref/1000, base='e')) + (s_7_6*pybamm.log(C_7ref/1000, base='e')) + (s_8_6*pybamm.log(C_8ref/1000, base='e')))

#Overpotentials for j reactions [V]
eta1=phi1-phi2-U1ref
eta2=phi1-phi2-U2ref
eta3=phi1-phi2-U3ref
eta4=phi1-phi2-U4ref
eta5=phi1-phi2-U5ref
eta6=phi1-phi2-U6ref

#Butler-Volmer equations for j reaction currents [A.m-2]
i1=i01ref*(((C1/C1ref)**(s11))*pybamm.exp((alpha_a1)*(F/(R*T))*eta1))
i2=i02ref*(((C2/C2ref)**(s22))*pybamm.exp((alpha_a2)*(F/(R*T))*eta2)-((C3/C3ref)**(-s32))*pybamm.exp((-alpha_c2)*(F/(R*T))*eta2))
i3=i03ref*(((C3/C3ref)**(s33))*pybamm.exp((alpha_a3)*(F/(R*T))*eta3)-((C4/C4ref)**(-s43))*pybamm.exp((-alpha_c3)*(F/(R*T))*eta3))
i4=i04ref*(((C4/C4ref)**(s44))*pybamm.exp((alpha_a4)*(F/(R*T))*eta4)-((C5/C5ref)**(-s54))*pybamm.exp((-alpha_c4)*(F/(R*T))*eta4))
i5=i05ref*(((C5/C5ref)**(s55))*pybamm.exp((alpha_a5)*(F/(R*T))*eta5)-((C6/C6ref)**(-s65))*pybamm.exp((-alpha_c5)*(F/(R*T))*eta5))
i6=i06ref*(((C6/C6ref)**(s66))*pybamm.exp((alpha_a6)*(F/(R*T))*eta6)-((C7/C7ref)**(-s76))*pybamm.exp((-alpha_c6)*(F/(R*T))*eta6))

#Production rate due to electrochemical reactions (r_i's)  [mol.m3.s-1]
r1=-a*((s11*i1/(n1*F))+(s12*i2/(n2*F))+(s13*i3/(n3*F))+(s14*i4/(n4*F))+(s15*i5/(n5*F))+(s16*i6/(n6*F)))
r2=-a*((s21*i1/(n1*F))+(s22*i2/(n2*F))+(s23*i3/(n3*F))+(s24*i4/(n4*F))+(s25*i5/(n5*F))+(s26*i6/(n6*F)))
r3=-a*((s31*i1/(n1*F))+(s32*i2/(n2*F))+(s33*i3/(n3*F))+(s34*i4/(n4*F))+(s35*i5/(n5*F))+(s36*i6/(n6*F)))
r4=-a*((s41*i1/(n1*F))+(s42*i2/(n2*F))+(s43*i3/(n3*F))+(s44*i4/(n4*F))+(s45*i5/(n5*F))+(s46*i6/(n6*F)))
r5=-a*((s51*i1/(n1*F))+(s52*i2/(n2*F))+(s53*i3/(n3*F))+(s54*i4/(n4*F))+(s55*i5/(n5*F))+(s56*i6/(n6*F)))
r6=-a*((s61*i1/(n1*F))+(s62*i2/(n2*F))+(s63*i3/(n3*F))+(s64*i4/(n4*F))+(s65*i5/(n5*F))+(s66*i6/(n6*F)))
r7=-a*((s71*i1/(n1*F))+(s72*i2/(n2*F))+(s73*i3/(n3*F))+(s74*i4/(n4*F))+(s75*i5/(n5*F))+(s76*i6/(n6*F)))
r8=-a*((s81*i1/(n1*F))+(s82*i2/(n2*F))+(s83*i3/(n3*F))+(s84*i4/(n4*F))+(s85*i5/(n5*F))+(s86*i6/(n6*F)))

#Flux (N_i_1 for separator, N_i_2 for positive electrode)
N1=epsilon*(-D1*pybamm.grad(C1)-z1*(D1/(R*T))*F*C1*pybamm.grad(phi2))
N2=epsilon*(-D2*pybamm.grad(C2)-z2*(D2/(R*T))*F*C2*pybamm.grad(phi2))
N3=epsilon*(-D3*pybamm.grad(C3)-z3*(D3/(R*T))*F*C3*pybamm.grad(phi2))
N4=epsilon*(-D4*pybamm.grad(C4)-z4*(D4/(R*T))*F*C4*pybamm.grad(phi2))
N5=epsilon*(-D5*pybamm.grad(C5)-z5*(D5/(R*T))*F*C5*pybamm.grad(phi2))
N6=epsilon*(-D6*pybamm.grad(C6)-z6*(D6/(R*T))*F*C6*pybamm.grad(phi2))
N7=epsilon*(-D7*pybamm.grad(C7)-z7*(D7/(R*T))*F*C7*pybamm.grad(phi2))
N8=epsilon*(-D8*pybamm.grad(C8)-z8*(D8/(R*T))*F*C8*pybamm.grad(phi2))

#Superficial current density in the liquid phase [A.m-2]
ie=F*(z1*N1+z2*N2+z3*N3+z4*N4+z5*N5+z6*N6+z7*N7+z8*N8)

#Rate of precipitation of precipitates (R'_k) [mol.m3.s-1]
Rdash1=k1*epsilon1*((C1**gamma11)*(C2**gamma21)*(C3**gamma31)*(C4**gamma41)*(C5**gamma51)*(C6**gamma61)*(C7**gamma71)*(C8**gamma81)-ksp1)
Rdash2=k2*epsilon2*((C1**gamma12)*(C2**gamma22)*(C3**gamma32)*(C4**gamma42)*(C5**gamma52)*(C6**gamma62)*(C7**gamma72)*(C8**gamma82)-ksp2)
Rdash3=k3*epsilon3*((C1**gamma13)*(C2**gamma23)*(C3**gamma33)*(C4**gamma43)*(C5**gamma53)*(C6**gamma63)*(C7**gamma73)*(C8**gamma83)-ksp3)
Rdash4=k4*epsilon4*((C1**gamma14)*(C2**gamma24)*(C3**gamma34)*(C4**gamma44)*(C5**gamma54)*(C6**gamma64)*(C7**gamma74)*(C8**gamma84)-ksp4)
Rdash5=k5*epsilon5*((C1**gamma15)*(C2**gamma25)*(C3**gamma35)*(C4**gamma45)*(C5**gamma55)*(C6**gamma65)*(C7**gamma75)*(C8**gamma85)-ksp5)

#Production rate of i species due to precipitation reactions (R_i) [mol.m3.s-1]
Rp1=gamma11*Rdash1+gamma12*Rdash2+gamma13*Rdash3+gamma14*Rdash4+gamma15*Rdash5
Rp2=gamma21*Rdash1+gamma22*Rdash2+gamma23*Rdash3+gamma24*Rdash4+gamma25*Rdash5
Rp3=gamma31*Rdash1+gamma32*Rdash2+gamma33*Rdash3+gamma34*Rdash4+gamma35*Rdash5
Rp4=gamma41*Rdash1+gamma42*Rdash2+gamma43*Rdash3+gamma44*Rdash4+gamma45*Rdash5
Rp5=gamma51*Rdash1+gamma52*Rdash2+gamma53*Rdash3+gamma54*Rdash4+gamma55*Rdash5
Rp6=gamma61*Rdash1+gamma62*Rdash2+gamma63*Rdash3+gamma64*Rdash4+gamma65*Rdash5
Rp7=gamma71*Rdash1+gamma72*Rdash2+gamma73*Rdash3+gamma74*Rdash4+gamma75*Rdash5
Rp8=gamma81*Rdash1+gamma82*Rdash2+gamma83*Rdash3+gamma84*Rdash4+gamma85*Rdash5

#Superficial current density in the solid phase [A.m-2]
is1=-sigma*pybamm.grad(phi1) #solid state current density exists only in the positive electrode

In [7]:
#Governing equations: (16 equations: 2 algebraic, 14 differential with 16 variables)
#1) Algebraic equations
model1.algebraic={
    phi2:pybamm.div(ie)-a*(i1+i2+i3+i4+i5+i6),
    phi1:pybamm.div(is1)+pybamm.div(ie),
}

#2) Differential equations
depsilon1dt=V1*Rdash1
depsilon2dt=V2*Rdash2
depsilon3dt=V3*Rdash3
depsilon4dt=V4*Rdash4
depsilon5dt=V5*Rdash5

depsilondt=-(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)

dC1dt=(C1*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N1)+r1-Rp1)/epsilon
dC2dt=(C2*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N2)+r2-Rp2)/epsilon
dC3dt=(C3*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N3)+r3-Rp3)/epsilon
dC4dt=(C4*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N4)+r4-Rp4)/epsilon
dC5dt=(C5*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N5)+r5-Rp5)/epsilon
dC6dt=(C6*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N6)+r6-Rp6)/epsilon
dC7dt=(C7*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N7)+r7-Rp7)/epsilon
dC8dt=(C8*(V1*Rdash1+V2*Rdash2+V3*Rdash3+V4*Rdash4+V5*Rdash5)-pybamm.div(N8)+r8-Rp8)/epsilon

model1.rhs={
    epsilon1:depsilon1dt, #1
    epsilon2:depsilon2dt,
    epsilon3:depsilon3dt,
    epsilon4:depsilon4dt,
    epsilon5:depsilon5dt, #5
    epsilon:depsilondt,
    C1:dC1dt,
    C2:dC2dt,
    C3:dC3dt,
    C4:dC4dt, #10
    C5:dC5dt,
    C6:dC6dt,
    C7:dC7dt,
    C8:dC8dt, #14
}

In [8]:
#Adding boundary conditions
model1.boundary_conditions = {
    phi1:{"left":(0,"Dirichlet"),"right":(-Iapp/sigma,"Neumann")},
    phi2:{"left":(0,"Dirichlet"),"right":(0,"Neumann")},
    C1:{"left":(-Iapp/(F*epsilon*D1),"Neumann"),"right":(0,"Neumann")},
    C2:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C3:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C4:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C5:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C6:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C7:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    C8:{"left":(0,"Neumann"),"right":(0,"Neumann")},
    epsilon:{"left":(0,"Dirichlet"),"right":(0,"Dirichlet")}
}

In [9]:
#Adding initial conditions
model1.initial_conditions = {
    phi1:U1ref+U2ref, #1
    phi2:-U1ref, #check
    epsilon1:0.160,
    epsilon2:1*10**(-6),
    epsilon3:1*10**(-6), #5
    epsilon4:1*10**(-6),
    epsilon5:1*10**(-7),
    epsilon:0.778,
    C1:1001.04,
    C2:19, #10
    C3:0.178,
    C4:0.324,
    C5:0.020,
    C6:5.229*10**(-7),
    C7:8.267*10**(-10), #15
    C8:1000, #16
}

In [10]:
#Adding variables in the model that could be directly called upon after solving the model

model1.variables={
    "Concentration of Li(+) [mol.m-3]":C1,
    "Concentration of S8(l) [mol.m-3]":C2,
    "Concentration of S8(2-) [mol.m-3]":C3,
    "Concentration of S6(2-) [mol.m-3]":C4,
    "Concentration of S4(2-) [mol.m-3]":C5,
    "Concentration of S2(2-) [mol.m-3]":C6,
    "Concentration of S(2-) [mol.m-3]":C7,
    "Concentration of A(-) [mol.m-3]":C8,
    "Porosity of positive electrode":epsilon,
    "Volume fraction of S8(s) precipitate in positive electrode":epsilon1,
    "Volume fraction of Li2S8(s) precipitate in positive electrode":epsilon2,
    "Volume fraction of Li2S4(s) precipitate in positive electrode":epsilon3,
    "Volume fraction of Li2S2(s) precipitate in positive electrode":epsilon4,
    "Volume fraction of Li2S(s) precipitate in positive electrode":epsilon5,
    "Liquid phase potential [V]": phi2,
}

In [11]:
#Defining functions for functional parameters D_i's and a
def Bruggemanexp1(epsilon):
    return D01*(epsilon**b)
def Bruggemanexp2(epsilon):
    return D02*(epsilon**b)
def Bruggemanexp3(epsilon):
    return D03*(epsilon**b)
def Bruggemanexp4(epsilon):
    return D04*(epsilon**b)
def Bruggemanexp5(epsilon):
    return D05*(epsilon**b)
def Bruggemanexp6(epsilon):
    return D06*(epsilon**b)
def Bruggemanexp7(epsilon):
    return D07*(epsilon**b)
def Bruggemanexp8(epsilon):
    return D08*(epsilon**b)

def specificsurfacearea(epsilon):
    return a0*(epsilon/epsilon0)**zeta

In [12]:
#Parameter values

param=pybamm.ParameterValues(
    {
        "Faraday's constant [C.mol-1]":96485.332, #F
        "Gas constant [J.K-1.mol-1]":8.31446, #R
        "Temperature [K]":293, #T (taken from the tanks-in-series model paper)
        "Bruggeman coefficient":1.5, #b
        "Initial value of specific area of positive electrode [m2.m-3]":132762, #a0
        "Morphology parameter":1.5, #zeta
        "Effective conductivity of solid phase of the positive electrode [S.m-1]":937, #sigma (taken from the tanks-in-series model paper)
        "Externally applied current [A.m-2]":0.394, #I_app (design parameter)
        
        "Charge number of Li(+)":1, #z_i
        "Charge number of S8(l)":0,
        "Charge number of S8(2-)":-2,
        "Charge number of S6(2-)":-2,
        "Charge number of S4(2-)":-2,
        "Charge number of S2(2-)":-2,
        "Charge number of S(2-)":-2,
        "Charge number of A(-)":-1,
        
        "Diffusion coefficient of Li(+) in bulk medium [m2.s-1]":1*10**(-10), #D0_i
        "Diffusion coefficient of S8(l) in bulk medium [m2.s-1]":1*10**(-9),
        "Diffusion coefficient of S8(2-) in bulk medium [m2.s-1]":6*10**(-10),
        "Diffusion coefficient of S6(2-) in bulk medium [m2.s-1]":6*10**(-10),
        "Diffusion coefficient of S4(2-) in bulk medium [m2.s-1]":1*10**(-10),
        "Diffusion coefficient of S2(2-) in bulk medium [m2.s-1]":1*10**(-10),
        "Diffusion coefficient of S(2-) in bulk medium [m2.s-1]":1*10**(-10),
        "Diffusion coefficient of A(-) in bulk medium [m2.s-1]":4*10**(-10),
        
        "Stoichiometric coefficient of Li(+) in reaction 1":-1, #s_i_j
        "Stoichiometric coefficient of Li(+) in reaction 2":0,
        "Stoichiometric coefficient of Li(+) in reaction 3":0,
        "Stoichiometric coefficient of Li(+) in reaction 4":0,
        "Stoichiometric coefficient of Li(+) in reaction 5":0,
        "Stoichiometric coefficient of Li(+) in reaction 6":0,
        "Stoichiometric coefficient of S8(l) in reaction 1":0,
        "Stoichiometric coefficient of S8(l) in reaction 2":-0.5,
        "Stoichiometric coefficient of S8(l) in reaction 3":0,
        "Stoichiometric coefficient of S8(l) in reaction 4":0,
        "Stoichiometric coefficient of S8(l) in reaction 5":0,
        "Stoichiometric coefficient of S8(l) in reaction 6":0,
        "Stoichiometric coefficient of S8(2-) in reaction 1":0,
        "Stoichiometric coefficient of S8(2-) in reaction 2":0.5,
        "Stoichiometric coefficient of S8(2-) in reaction 3":-1.5,
        "Stoichiometric coefficient of S8(2-) in reaction 4":0,
        "Stoichiometric coefficient of S8(2-) in reaction 5":0,
        "Stoichiometric coefficient of S8(2-) in reaction 6":0,
        "Stoichiometric coefficient of S6(2-) in reaction 1":0,
        "Stoichiometric coefficient of S6(2-) in reaction 2":0,
        "Stoichiometric coefficient of S6(2-) in reaction 3":2,
        "Stoichiometric coefficient of S6(2-) in reaction 4":-1,
        "Stoichiometric coefficient of S6(2-) in reaction 5":0,
        "Stoichiometric coefficient of S6(2-) in reaction 6":0,
        "Stoichiometric coefficient of S4(2-) in reaction 1":0,
        "Stoichiometric coefficient of S4(2-) in reaction 2":0,
        "Stoichiometric coefficient of S4(2-) in reaction 3":0,
        "Stoichiometric coefficient of S4(2-) in reaction 4":1.5,
        "Stoichiometric coefficient of S4(2-) in reaction 5":-0.5,
        "Stoichiometric coefficient of S4(2-) in reaction 6":0,
        "Stoichiometric coefficient of S2(2-) in reaction 1":0,
        "Stoichiometric coefficient of S2(2-) in reaction 2":0,
        "Stoichiometric coefficient of S2(2-) in reaction 3":0,
        "Stoichiometric coefficient of S2(2-) in reaction 4":0,
        "Stoichiometric coefficient of S2(2-) in reaction 5":1,
        "Stoichiometric coefficient of S2(2-) in reaction 6":-0.5,
        "Stoichiometric coefficient of S(2-) in reaction 1":0,
        "Stoichiometric coefficient of S(2-) in reaction 2":0,
        "Stoichiometric coefficient of S(2-) in reaction 3":0,
        "Stoichiometric coefficient of S(2-) in reaction 4":0,
        "Stoichiometric coefficient of S(2-) in reaction 5":0,
        "Stoichiometric coefficient of S(2-) in reaction 6":1,
        "Stoichiometric coefficient of A(-) in reaction 1":0,
        "Stoichiometric coefficient of A(-) in reaction 2":0,
        "Stoichiometric coefficient of A(-) in reaction 3":0,
        "Stoichiometric coefficient of A(-) in reaction 4":0,
        "Stoichiometric coefficient of A(-) in reaction 5":0,
        "Stoichiometric coefficient of A(-) in reaction 6":0,
        
        "Number of electrons transferred in reaction 1":1, #n_j
        "Number of electrons transferred in reaction 2":1,
        "Number of electrons transferred in reaction 3":1,
        "Number of electrons transferred in reaction 4":1,
        "Number of electrons transferred in reaction 5":1,
        "Number of electrons transferred in reaction 6":1,
        
        "Initial value of porosity of positive electrode":0.778,
        
        "Exchange current density of reaction 1 at reference concentrations [A.m-2]":0.394, #i0_jref
        "Exchange current density of reaction 2 at reference concentrations [A.m-2]":1.972,
        "Exchange current density of reaction 3 at reference concentrations [A.m-2]":0.019,
        "Exchange current density of reaction 4 at reference concentrations [A.m-2]":0.019,
        "Exchange current density of reaction 5 at reference concentrations [A.m-2]":1.97*10**(-4),
        "Exchange current density of reaction 6 at reference concentrations [A.m-2]":1.97*10**(-7),
        
        "Reference concentration of Li(+) [mol.m-3]":1001.04, #C_iref
        "Reference concentration of S8(l) [mol.m-3]":19.0,
        "Reference concentration of S8(2-) [mol.m-3]":0.178,
        "Reference concentration of S6(2-) [mol.m-3]":0.324,
        "Reference concentration of S4(2-) [mol.m-3]":0.020,
        "Reference concentration of S2(2-) [mol.m-3]":5.229*10**(-7),
        "Reference concentration of S(2-) [mol.m-3]":8.267*10**(-10),
        "Reference concentration of A(-) [mol.m-3]":1000,
        
        "Anodic transfer coefficient of reaction 1":0.5, #alpha_aj, alpha_cj
        "Anodic transfer coefficient of reaction 2":0.5,
        "Anodic transfer coefficient of reaction 3":0.5,
        "Anodic transfer coefficient of reaction 4":0.5,
        "Anodic transfer coefficient of reaction 5":0.5,
        "Anodic transfer coefficient of reaction 6":0.5,
        "Cathodic transfer coefficient of reaction 1":0.5,
        "Cathodic transfer coefficient of reaction 2":0.5,
        "Cathodic transfer coefficient of reaction 3":0.5,
        "Cathodic transfer coefficient of reaction 4":0.5,
        "Cathodic transfer coefficient of reaction 5":0.5,
        "Cathodic transfer coefficient of reaction 6":0.5,
        
        "Standard OCP of reaction 1 [V]":0, #Utheta_j
        "Standard OCP of reaction 2 [V]":2.39,
        "Standard OCP of reaction 3 [V]":2.37,
        "Standard OCP of reaction 4 [V]":2.24,
        "Standard OCP of reaction 5 [V]":2.04,
        "Standard OCP of reaction 6 [V]":2.01,
        
        "Number of Li(+) produced by dissociation of S8(s)":0, #gamma_i_k
        "Number of Li(+) produced by dissociation of Li2S8(s)":2,
        "Number of Li(+) produced by dissociation of Li2S4(s)":2,
        "Number of Li(+) produced by dissociation of Li2S2(s)":2,
        "Number of Li(+) produced by dissociation of Li2S(s)":2,
        "Number of S8(l) produced by dissociation of S8(s)":1,
        "Number of S8(l) produced by dissociation of Li2S8(s)":0,
        "Number of S8(l) produced by dissociation of Li2S4(s)":0,
        "Number of S8(l) produced by dissociation of Li2S2(s)":0,
        "Number of S8(l) produced by dissociation of Li2S(s)":0,
        "Number of S8(2-) produced by dissociation of S8(s)":0,
        "Number of S8(2-) produced by dissociation of Li2S8(s)":1,
        "Number of S8(2-) produced by dissociation of Li2S4(s)":0,
        "Number of S8(2-) produced by dissociation of Li2S2(s)":0,
        "Number of S8(2-) produced by dissociation of Li2S(s)":0,
        "Number of S6(2-) produced by dissociation of S8(s)":0,
        "Number of S6(2-) produced by dissociation of Li2S8(s)":0,
        "Number of S6(2-) produced by dissociation of Li2S4(s)":0,
        "Number of S6(2-) produced by dissociation of Li2S2(s)":0,
        "Number of S6(2-) produced by dissociation of Li2S(s)":0,
        "Number of S4(2-) produced by dissociation of S8(s)":0,
        "Number of S4(2-) produced by dissociation of Li2S8(s)":0,
        "Number of S4(2-) produced by dissociation of Li2S4(s)":1,
        "Number of S4(2-) produced by dissociation of Li2S2(s)":0,
        "Number of S4(2-) produced by dissociation of Li2S(s)":0,
        "Number of S2(2-) produced by dissociation of S8(s)":0,
        "Number of S2(2-) produced by dissociation of Li2S8(s)":0,
        "Number of S2(2-) produced by dissociation of Li2S4(s)":0,
        "Number of S2(2-) produced by dissociation of Li2S2(s)":1,
        "Number of S2(2-) produced by dissociation of Li2S(s)":0,
        "Number of S(2-) produced by dissociation of S8(s)":0,
        "Number of S(2-) produced by dissociation of Li2S8(s)":0,
        "Number of S(2-) produced by dissociation of Li2S4(s)":0,
        "Number of S(2-) produced by dissociation of Li2S2(s)":0,
        "Number of S(2-) produced by dissociation of Li2S(s)":1,
        "Number of A(-) produced by dissociation of S8(s)":0, #Potential check for A(-), although this shouldn't be an issue
        "Number of A(-) produced by dissociation of Li2S8(s)":0,
        "Number of A(-) produced by dissociation of Li2S4(s)":0,
        "Number of A(-) produced by dissociation of Li2S2(s)":0,
        "Number of A(-) produced by dissociation of Li2S(s)":0,
        
        "Rate constant of S8(s) [s-1]":1.0, #k_k
        "Rate constant of Li2S8(s) [m6.mol2.s-1]":1*10**(-4),
        "Rate constant of Li2S4(s) [m6.mol2.s-1]":9.98*10**(-5),
        "Rate constant of Li2S2(s) [m6.mol2.s-1]":9.98*10**(-4),
        "Rate constant of Li2S(s) [m6.mol2.s-1]":27.5,
        
        "Solubility product of S8(s) [mol.m-3]":19.0, #ksp_k
        "Solubility product of Li2S8(s) [mol3.m-9]":38.09,
        "Solubility product of Li2S4(s) [mol3.m-9]":11.26,
        "Solubility product of Li2S2(s) [mol3.m-9]":5.1*10**(-3),
        "Solubility product of Li2S(s) [mol3.m-9]":3.0*10**(-5),
        
        "Molar volume of S8(s) [m3.mol-1]":1.239*10**(-4), #V_k
        "Molar volume of Li2S8(s) [m3.mol-1]":1.361*10**(-4),
        "Molar volume of Li2S4(s) [m3.mol-1]":7.415*10**(-5),
        "Molar volume of Li2S2(s) [m3.mol-1]":4.317*10**(-5),
        "Molar volume of Li2S(s) [m3.mol-1]":2.768*10**(-5),
        
        "Thickness of positive electrode [m]":41*10**(-6), #L_c
        
        "Specific surface area of positive electrode [m2.m-3]":specificsurfacearea, #a
        
        "Diffusion coefficient of Li(+) in porous medium in positive electrode [m2.s-1]": Bruggemanexp1, #D_i_2's
        "Diffusion coefficient of S8(l) in porous medium in positive electrode [m2.s-1]": Bruggemanexp2,
        "Diffusion coefficient of S8(2-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp3,
        "Diffusion coefficient of S6(2-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp4,
        "Diffusion coefficient of S4(2-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp5,
        "Diffusion coefficient of S2(2-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp6,
        "Diffusion coefficient of S(2-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp7,
        "Diffusion coefficient of A(-) in porous medium in positive electrode [m2.s-1]": Bruggemanexp8,
        
        "Reference OCP of reaction 1 [V]":Utheta1-R*T/(n1*F)*((s11*pybamm.log(C1ref/1000,base='e'))+(s21*pybamm.log(C2ref/1000,base='e'))+(s31*pybamm.log(C3ref/1000,base='e'))+(s41*pybamm.log(C4ref/1000,base='e'))+(s51*pybamm.log(C5ref/1000,base='e'))+(s61*pybamm.log(C6ref/1000,base='e'))+(s71*pybamm.log(C7ref/1000,base='e'))+(s81*pybamm.log(C8ref/1000,base='e'))),
        "Reference OCP of reaction 2 [V]":Utheta2-R*T/(n2*F)*((s12*pybamm.log(C1ref/1000,base='e'))+(s22*pybamm.log(C2ref/1000,base='e'))+(s32*pybamm.log(C3ref/1000,base='e'))+(s42*pybamm.log(C4ref/1000,base='e'))+(s52*pybamm.log(C5ref/1000,base='e'))+(s62*pybamm.log(C6ref/1000,base='e'))+(s72*pybamm.log(C7ref/1000,base='e'))+(s82*pybamm.log(C8ref/1000,base='e'))),
        "Reference OCP of reaction 3 [V]":Utheta3-R*T/(n3*F)*((s13*pybamm.log(C1ref/1000,base='e'))+(s23*pybamm.log(C2ref/1000,base='e'))+(s33*pybamm.log(C3ref/1000,base='e'))+(s43*pybamm.log(C4ref/1000,base='e'))+(s53*pybamm.log(C5ref/1000,base='e'))+(s63*pybamm.log(C6ref/1000,base='e'))+(s73*pybamm.log(C7ref/1000,base='e'))+(s83*pybamm.log(C8ref/1000,base='e'))),
        "Reference OCP of reaction 4 [V]":Utheta4-R*T/(n4*F)*((s14*pybamm.log(C1ref/1000,base='e'))+(s24*pybamm.log(C2ref/1000,base='e'))+(s34*pybamm.log(C3ref/1000,base='e'))+(s44*pybamm.log(C4ref/1000,base='e'))+(s54*pybamm.log(C5ref/1000,base='e'))+(s64*pybamm.log(C6ref/1000,base='e'))+(s74*pybamm.log(C7ref/1000,base='e'))+(s84*pybamm.log(C8ref/1000,base='e'))),
        "Reference OCP of reaction 5 [V]":Utheta5-R*T/(n5*F)*((s15*pybamm.log(C1ref/1000,base='e'))+(s25*pybamm.log(C2ref/1000,base='e'))+(s35*pybamm.log(C3ref/1000,base='e'))+(s45*pybamm.log(C4ref/1000,base='e'))+(s55*pybamm.log(C5ref/1000,base='e'))+(s65*pybamm.log(C6ref/1000,base='e'))+(s75*pybamm.log(C7ref/1000,base='e'))+(s85*pybamm.log(C8ref/1000,base='e'))),
        "Reference OCP of reaction 6 [V]":Utheta6-R*T/(n6*F)*((s16*pybamm.log(C1ref/1000,base='e'))+(s26*pybamm.log(C2ref/1000,base='e'))+(s36*pybamm.log(C3ref/1000,base='e'))+(s46*pybamm.log(C4ref/1000,base='e'))+(s56*pybamm.log(C5ref/1000,base='e'))+(s66*pybamm.log(C6ref/1000,base='e'))+(s76*pybamm.log(C7ref/1000,base='e'))+(s86*pybamm.log(C8ref/1000,base='e'))),
    }
)



In [13]:
#Defining geometry
xp=pybamm.SpatialVariable("xp",domain=["positive electrode"],coord_sys="cartesian")

geometry={
    "positive electrode":{xp:{"min":0,"max":Lc}},
}

In [14]:
#Processing the model with input parameter values
param.process_model(model1)

2022-06-15 15:28:28,193 - [INFO] parameter_values.process_model(415): Start setting parameters for 1D LiS (without separator)
2022-06-15 15:28:28,197 - [VERBOSE] parameter_values.process_model(437): Processing parameters for Variable(-0x4665b4d000002242, Volume fraction of S8(s) precipitate in positive electrode, children=[], domain=['positive electrode'], auxiliary_domains={}) (rhs)
2022-06-15 15:28:28,200 - [VERBOSE] parameter_values.process_model(437): Processing parameters for Variable(0x3a47e01b143f1c10, Volume fraction of Li2S8(s) precipitate in positive electrode, children=[], domain=['positive electrode'], auxiliary_domains={}) (rhs)
2022-06-15 15:28:28,204 - [VERBOSE] parameter_values.process_model(437): Processing parameters for Variable(-0x104b30c1aeaf73a5, Volume fraction of Li2S4(s) precipitate in positive electrode, children=[], domain=['positive electrode'], auxiliary_domains={}) (rhs)
2022-06-15 15:28:28,208 - [VERBOSE] parameter_values.process_model(437): Processing pa

2022-06-15 15:28:28,316 - [VERBOSE] parameter_values.process_model(453): Processing parameters for Variable(-0x6e90152a613ad35d, Concentration of A(-) in positive electrode [mol.m-3], children=[], domain=['positive electrode'], auxiliary_domains={}) (initial conditions)
2022-06-15 15:28:28,317 - [VERBOSE] parameter_values.process_boundary_conditions(526): Processing parameters for Variable(-0x545a6c8c04fd1434, Solid phase potential in positive electrode [V], children=[], domain=['positive electrode'], auxiliary_domains={}) (left bc)
2022-06-15 15:28:28,317 - [VERBOSE] parameter_values.process_boundary_conditions(526): Processing parameters for Variable(-0x545a6c8c04fd1434, Solid phase potential in positive electrode [V], children=[], domain=['positive electrode'], auxiliary_domains={}) (right bc)
2022-06-15 15:28:28,318 - [VERBOSE] parameter_values.process_boundary_conditions(526): Processing parameters for Variable(0x1fa4b6e771648e23, Liquid phase potential in positive electrode [V], 

2022-06-15 15:28:28,355 - [VERBOSE] parameter_values.process_model(463): Processing parameters for 'Volume fraction of Li2S(s) precipitate in positive electrode' (variables)
2022-06-15 15:28:28,357 - [VERBOSE] parameter_values.process_model(463): Processing parameters for 'Liquid phase potential [V]' (variables)
2022-06-15 15:28:28,359 - [INFO] parameter_values.process_model(505): Finish setting parameters for 1D LiS (without separator)


In [15]:
##Processing the chosen geometry with input parameter values
param.process_geometry(geometry)

In [16]:
submesh_types={
    "positive electrode":pybamm.MeshGenerator(pybamm.Uniform1DSubMesh),
}
var_pts={xp:4}
mesh=pybamm.Mesh(geometry,submesh_types,var_pts)

spatial_methods={
    "positive electrode":pybamm.FiniteVolume()
}
disc=pybamm.Discretisation(mesh,spatial_methods)
disc.process_model(model1)

2022-06-15 15:28:28,432 - [INFO] discretisation.process_model(137): Start discretising 1D LiS (without separator)
2022-06-15 15:28:28,445 - [VERBOSE] discretisation.process_model(161): Set variable slices for 1D LiS (without separator)
2022-06-15 15:28:28,446 - [VERBOSE] discretisation.process_model(171): Discretise boundary conditions for 1D LiS (without separator)
2022-06-15 15:28:28,447 - [DEBUG] discretisation.process_boundary_conditions(524): Discretise Solid phase potential in positive electrode [V] (left bc)
2022-06-15 15:28:28,448 - [DEBUG] discretisation.process_boundary_conditions(524): Discretise Solid phase potential in positive electrode [V] (right bc)
2022-06-15 15:28:28,448 - [DEBUG] discretisation.process_boundary_conditions(524): Discretise Liquid phase potential in positive electrode [V] (left bc)
2022-06-15 15:28:28,449 - [DEBUG] discretisation.process_boundary_conditions(524): Discretise Liquid phase potential in positive electrode [V] (right bc)
2022-06-15 15:28:28

2022-06-15 15:28:28,489 - [DEBUG] discretisation.process_dict(802): Discretise 'Concentration of S6(2-) [mol.m-3]'
2022-06-15 15:28:28,491 - [DEBUG] discretisation.process_dict(802): Discretise 'Concentration of S4(2-) [mol.m-3]'
2022-06-15 15:28:28,493 - [DEBUG] discretisation.process_dict(802): Discretise 'Concentration of S2(2-) [mol.m-3]'
2022-06-15 15:28:28,498 - [DEBUG] discretisation.process_dict(802): Discretise 'Concentration of S(2-) [mol.m-3]'
2022-06-15 15:28:28,500 - [DEBUG] discretisation.process_dict(802): Discretise 'Concentration of A(-) [mol.m-3]'
2022-06-15 15:28:28,501 - [DEBUG] discretisation.process_dict(802): Discretise 'Porosity of positive electrode'
2022-06-15 15:28:28,501 - [DEBUG] discretisation.process_dict(802): Discretise 'Volume fraction of S8(s) precipitate in positive electrode'
2022-06-15 15:28:28,502 - [DEBUG] discretisation.process_dict(802): Discretise 'Volume fraction of Li2S8(s) precipitate in positive electrode'
2022-06-15 15:28:28,502 - [DEBUG]

ValueError: dimension mismatch

In [ ]:
solver=pybamm.CasadiSolver(atol=1e-8, rtol=1e-8)
t_eval=np.linspace(0, 3600, 600)
solution=solver.solve(model1, t_eval)